In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

50 documents
9 classes ['Additional_Benefits', 'Car_price', 'Discounts', 'Features', 'Models', 'Return_Policies', 'goodbye', 'greeting', 'thanks']
107 unique lemmatized words ["'s", ',', 'a', 'additional', 'all', 'and', 'any', 'anyone', 'anything', 'are', 'available', 'awesome', 'back', 'benefit', 'bill', 'bit', 'bye', 'can', 'car', 'chatting', 'cost', 'costly', 'could', 'currently', 'day', 'decrease', 'discount', 'do', 'doe', 'else', 'expect', 'extra', 'feature', 'find', 'fixed', 'flaw', 'for', 'from', 'get', 'give', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'if', 'in', 'info/details', 'is', 'kind', 'later', 'like', 'live', 'me', 'model', 'money/amount', 'more', 'much', 'my', "n't", 'new', 'next', 'nice', 'of', 'off', 'offering', 'on', 'option', 'part', 'pay', 'policy', 'price', 'price/amount', 'price/cost/bill', 'properly/perfectly/smoothly/satisfactorily/nicely/', 'return', 'see', 'should', 'some', 'speciality', 'ssup', 'thank', 'than

In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [8]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

In [9]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [12]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2658 - accuracy: 0.0600
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0767 - accuracy: 0.2400
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0354 - accuracy: 0.2200
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9522 - accuracy: 0.3200
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7967 - accuracy: 0.3400
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.6630 - accuracy: 0.5200
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4557 - accuracy: 0.5400
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3587 - accuracy: 0.6400
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.1147 - accuracy: 0.7800
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0466 - accuracy: 0.7200
Epoch 11/

10/10 [==============================] - 0s 2ms/step - loss: 0.0559 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0781 - accuracy: 0.9800
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0237 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0837 - accuracy: 0.9800
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0394 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0720 - accuracy: 0.9800
Epoch 90/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0342 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0830 - accuracy: 0.9800
Epoch 92/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0245 - accuracy: 1.0000
Epoch 93/200


10/10 [==============================] - 0s 1ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0320 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0403 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0214 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0344 - accuracy: 0.9800
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0119 - accuracy: 1.0000
Epoc

In [13]:
#--------------------------------------------------------Inference part--------------------------------------------------------#

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

In [15]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [16]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


In [17]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [18]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [19]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [20]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [33]:
msg = 'what is the cost of model x'
res = chatbot_response(msg)

In [34]:
res

'Price of a new car : Rs X, Price of a used car: Rs Y.  Payment can be either through cash or card'